## Tech Challenge

### O que é o projeto

Entender como foi o comportamento da população na época da pandemia da COVID-19 e quais indicadores seriam importantes para o planejamento, caso haja um novo surto da doença, utilizando o estudo do PNAD-COVID 19 do IBGE para termos respostas ao problema proposto, pois são dados confiáveis, porém, não será necessário utilizar todas as perguntas realizadas na pesquisa para enxergar todas as oportunidades ali postas, mas há dados triviais que precisam estar no projeto, pois auxiliam muito na análise dos dados:

• Características clínicas dos sintomas;  
• Características da população;  
• Características econômicas da sociedade.  

Dessa forma, acessar os dados do PNAD-COVID-19 do IBGE (https://covid19.ibge.gov.br/pnad-covid/) e organizar esta base para análise, utilizando Banco de Dados em Nuvem e trazendo as seguintes características:

a. Utilização de no máximo 20 questionamentos realizados na pesquisa;  
b. Utilizar 3 meses para construção da solução;  
c. Caracterização dos sintomas clínicos da população;  
d. Comportamento da população na época da COVID-19;  
e. Características econômicas da Sociedade;  

Com objetivo de trazer uma breve análise dessas informações, como foi a organização do banco, as perguntas selecionadas para a resposta do problema e quais seriam as principais ações que o hospital deverá tomar em caso de um novo surto de COVID-19

### Arquitetura de Dados

![Arquitetura](arquitetura.png)

1) Dados referente a Pesquisa Nacional por Amostra de Domicílios (PNAD COVID19) através do link [PNAD](https://www.ibge.gov.br/estatisticas/investigacoes-experimentais/estatisticas-experimentais/27946-divulgacao-semanal-pnadcovid1?t=microdados&utm_source=covid19&utm_medium=hotsite&utm_campaign=covid_19) *
2) Realizado o processo de ETL para extrair os dados (E), tratar os dados (T) e carregar os dados (L)
3) Dados armazenado em um banco dados PostgreSQL criado via AWS RDS 
4) Aplicar as regras de negócio como filtro das perguntas necessarias e dados dos ultimos 3 meses
5) Realizar o carregamento dos dados em um tabela especifica (data mart) esse disponiviel no PostgreSQL 
6) Construir a analise utilizando o Power BI conectado ao Data Mart


\* Os dados foram armazenados no GitHub para a construção desse trabalho

### Códigos

#### Importar bibliotecas

In [1]:
# Importar biblioteca completa
import requests
import zipfile
import io
import pandas as pd
import os
import boto3

# Importar algo especifico de uma biblioteca
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from botocore.exceptions import BotoCoreError, ClientError

#### Funções (DEF)

In [2]:
# Testar a conexão ao banco de dados
def test_connection(engine):

    try:
        with engine.connect() as connection:
            
            # Testar a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Listar as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'public';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("  -", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)


#### Validar Conexões e criar engine banco de dados

In [3]:
# Validar conexão com a AWS --> credentials
try:
    session = boto3.Session(profile_name="academy")
    sts = session.client("sts")
    identity = sts.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)


✅ Conectado à conta

UserId: AROAWIUHO6CVULDIIR3FL:user4308167=ricardviana1@gmail.com
Account: 430854566059
Arn: arn:aws:sts::430854566059:assumed-role/voclabs/user4308167=ricardviana1@gmail.com


In [4]:
# Validar conexão com a AWS --> .env
load_dotenv()

try:
    sts_client = boto3.client(
        'sts',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
        region_name=os.getenv('AWS_REGION')
    )
    
    session = boto3.Session(profile_name="academy")
    sts = session.client("sts")
    identity = sts_client.get_caller_identity()
    print("✅ Conectado à conta\n")
    print("UserId:", identity["UserId"])
    print("Account:", identity["Account"])
    print("Arn:", identity["Arn"])

except (BotoCoreError, ClientError) as e:
    print("❌ Erro ao conectar à AWS. Verifique suas credenciais e tente novamente.")
    print("Detalhes do erro:", e)


✅ Conectado à conta

UserId: AROAWIUHO6CVULDIIR3FL:user4308167=ricardviana1@gmail.com
Account: 430854566059
Arn: arn:aws:sts::430854566059:assumed-role/voclabs/user4308167=ricardviana1@gmail.com


In [5]:
# Criar a engine para conexão ao banco de dados usando
load_dotenv()

usuario = os.getenv("POSTGRES_USER_PNAD")
senha = os.getenv("POSTGRES_PASSWORD_PNAD")
host = os.getenv("POSTGRES_HOST_PNAD")
porta = os.getenv("POSTGRES_PORT_PNAD")
banco = os.getenv("POSTGRES_DB_PNAD")

engine = create_engine(f"postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco}")

# Testar a conexão
test_connection(engine)

✅ Conectado com sucesso: PostgreSQL 17.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit
📄 Tabelas no banco:
  - questionario_pnad_covid


#### Realizar o processo de ETL

In [ ]:
api_url = 'https://api.github.com/repos/RicardViana/fiap-Big-Data/contents/PNAD-COVID/Microdados'

# Nome do seu bucket S3 e da subpasta de destino (SEPARADOS)
s3_bucket = 'tech-challenge-430854566059'
s3_subpasta = 'bronze'

# Inicializa o cliente do S3
s3_client = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
    region_name=os.getenv('AWS_REGION')
)

print(f"Buscando lista de arquivos em: {api_url}")
response = requests.get(api_url)
response.raise_for_status()
files = response.json()

# Itera sobre cada item no diretório do GitHub
for file_info in files:

    if file_info['name'].endswith('.zip'):
        zip_name = file_info['name']
        zip_url = file_info['download_url']
        
        print(f"\nProcessando: {zip_name}")
        
        try:
            r_zip = requests.get(zip_url)
            r_zip.raise_for_status()
            zip_content = io.BytesIO(r_zip.content)

            with zipfile.ZipFile(zip_content) as z:
                csv_filename = [f for f in z.namelist() if f.endswith('.csv')][0]
                print(f"Arquivo CSV encontrado: {csv_filename}")

                csv_content_bytes = z.read(csv_filename)
        
                caminho_completo_s3 = f"{s3_subpasta}/{csv_filename}"
                
                print(f"Enviando para o S3 em: '{caminho_completo_s3}'...")
                s3_client.put_object(
                    Bucket=s3_bucket,
                    Key=caminho_completo_s3,
                    Body=csv_content_bytes
                )
                print(f"✅ Sucesso! Arquivo enviado para s3://{s3_bucket}/{caminho_completo_s3}")
        
        except Exception as e:
            print(f"ERRO: Ocorreu um erro inesperado ao processar {zip_name}. Erro: {e}")

print("\nProcesso de ETL para o S3 concluído!")

Buscando lista de arquivos em: https://api.github.com/repos/RicardViana/fiap-Big-Data/contents/PNAD-COVID/Microdados

Processando: PNAD_COVID_052020.zip ---
Arquivo CSV encontrado: PNAD_COVID_052020.csv
Enviando para o S3 em: 'bronze/PNAD_COVID_052020.csv'...
✅ Sucesso! Arquivo enviado para s3://tech-challenge-430854566059/bronze/PNAD_COVID_052020.csv

Processando: PNAD_COVID_062020.zip ---
Arquivo CSV encontrado: PNAD_COVID_062020.csv
Enviando para o S3 em: 'bronze/PNAD_COVID_062020.csv'...
✅ Sucesso! Arquivo enviado para s3://tech-challenge-430854566059/bronze/PNAD_COVID_062020.csv

Processando: PNAD_COVID_072020.zip ---
Arquivo CSV encontrado: PNAD_COVID_072020.csv
Enviando para o S3 em: 'bronze/PNAD_COVID_072020.csv'...
✅ Sucesso! Arquivo enviado para s3://tech-challenge-430854566059/bronze/PNAD_COVID_072020.csv

Processando: PNAD_COVID_082020.zip ---
Arquivo CSV encontrado: PNAD_COVID_082020.csv
Enviando para o S3 em: 'bronze/PNAD_COVID_082020.csv'...
✅ Sucesso! Arquivo enviado pa

In [ ]:
# Carrega as variáveis do arquivo .env
load_dotenv()

# --- 1. CONFIGURAÇÕES ---
# URL do arquivo CSV a ser baixado
url_csv_uf = "https://raw.githubusercontent.com/RicardViana/tabela-uf-ibge/refs/heads/main/codigo_uf.csv"

# Configurações do seu S3
s3_bucket = 'tech-challenge-430854566059'
s3_subpasta = 'bronze'
nome_arquivo_s3 = 'codigo_uf.csv' # Nome que o arquivo terá no S3

# Monta o caminho completo no S3
caminho_completo_s3 = f"{s3_subpasta}/{nome_arquivo_s3}"

# --- 2. PROCESSO DE UPLOAD ---
try:
    # Passo 1: Baixar o conteúdo do arquivo da URL para a memória
    print(f"Baixando o arquivo CSV de: {url_csv_uf}")
    response = requests.get(url_csv_uf)
    response.raise_for_status()  # Verifica se o download foi bem-sucedido
    
    # O conteúdo do arquivo já vem em 'bytes', pronto para o upload
    csv_content_bytes = response.content

    # Passo 2: Inicializar o cliente do S3
    print("Conectando ao S3...")
    s3_client = boto3.client(
        's3',
        aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
        aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
        aws_session_token=os.getenv('AWS_SESSION_TOKEN'),
        region_name=os.getenv('AWS_REGION')
    )

    # Passo 3: Enviar o conteúdo para o S3
    print(f"Enviando '{nome_arquivo_s3}' para s3://{s3_bucket}/{s3_subpasta}/...")
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=caminho_completo_s3,
        Body=csv_content_bytes
    )

    print(f"✅ Sucesso! Arquivo '{nome_arquivo_s3}' carregado na camada Bronze.")

except requests.exceptions.RequestException as e:
    print(f"❌ ERRO: Falha ao baixar o arquivo da URL. Erro: {e}")
except (BotoCoreError, ClientError) as e:
    print(f"❌ ERRO: Falha ao conectar ou enviar o arquivo para o S3. Verifique suas credenciais. Erro: {e}")
except Exception as e:
    print(f"❌ ERRO: Ocorreu um erro inesperado. Erro: {e}")

In [ ]:
# Carrega as variáveis do arquivo .env
load_dotenv()

# --- 1. CONFIGURAÇÃO DE CREDENCIAIS E CAMINHOS ---

# Opções de armazenamento para o Pandas usar as credenciais do Boto3
# (Adaptado para usar as credenciais do .env que já configuramos)
storage_options = {
    "key": os.getenv('AWS_ACCESS_KEY_ID'),
    "secret": os.getenv('AWS_SECRET_ACCESS_KEY'),
    "token": os.getenv('AWS_SESSION_TOKEN')
}

# Definição dos seus caminhos (prefixos) no S3
BRONZE_PREFIX = 's3://tech-challenge-430854566059/bronze/'
LOOKUP_PREFIX = 's3://tech-challenge-430854566059/lookup/'
SILVER_PREFIX = 's3://tech-challenge-430854566059/silver/'

# --- 2. LEITURA E CONSOLIDAÇÃO DA CAMADA BRONZE ---

# Listar todos os arquivos CSV na pasta bronze
# Para isso, usamos o Boto3
s3_client = boto3.client('s3', **storage_options)
bucket_name = 'tech-challenge-430854566059'
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='bronze/')
bronze_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.csv')]

# Ler cada arquivo CSV do S3 e consolidar
lista_de_dataframes = []
print("Lendo arquivos da camada Bronze no S3...")
for file_key in bronze_files:
    file_path = f"s3://{bucket_name}/{file_key}"
    print(f"  - Lendo {file_path}")
    df_temp = pd.read_csv(file_path, storage_options=storage_options, sep=',')
    lista_de_dataframes.append(df_temp)

df_consolidado = pd.concat(lista_de_dataframes, ignore_index=True)
print(f"\nConsolidação concluída. {len(df_consolidado)} linhas lidas da camada Bronze.")

# --- 3. LEITURA DA TABELA DE LOOKUP ---

link_codigo_uf = f"{LOOKUP_PREFIX}codigo_uf.csv"
print(f"Lendo tabela de lookup de: {link_codigo_uf}")
df_uf = pd.read_csv(link_codigo_uf, sep=",", storage_options=storage_options)

# --- 4. TRANSFORMAÇÃO E ENRIQUECIMENTO (SUA LÓGICA) ---

print("Iniciando enriquecimento dos dados (merge com UF)...")
# Relacionar os dados data frame (exatamente como você fez)
df_silver = pd.merge(
    df_consolidado,
    df_uf,
    how='left',
    left_on='UF', 
    right_on='Código'
)

# Remover colunas não necessárias e renomear
df_silver = df_silver.drop(columns=["Código"])
df_silver = df_silver.rename(columns={"UF_x": "UF", "UF_y": "UF_Nome"})
print("Enriquecimento concluído.")

# --- 5. SALVAR A CAMADA SILVER NO S3 ---

caminho_saida_silver = SILVER_PREFIX + 'pnad_consolidado_enriquecido.parquet'
print(f"\nSalvando dados da camada Silver em: {caminho_saida_silver}")

# Salvar o DataFrame transformado como um arquivo Parquet no S3
df_silver.to_parquet(
    caminho_saida_silver,
    index=False,
    storage_options=storage_options
)

print("✅ Sucesso! Camada Silver criada e salva no S3.")